In [1]:
'''Code for fine-tuning Inception V3 for a new task.

Start with Inception V3 network, not including last fully connected layers.

Train a simple fully connected layer on top of these.


'''

import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout
import inception_v3 as inception
import vgg16 as VGG
'''
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))
'''
N_CLASSES = 2
IMSIZE = (224, 224)

# TO DO:: Replace these with paths to the downloaded data.
# Training directory
train_dir = '../data/catdog/train'
# Testing directory
test_dir = '../data/catdog/validation'


# Start with an Inception V3 model, not including the final softmax layer.
base_model = VGG.VGG16(weights='imagenet')
print ('Loaded vgg16 model')

Using TensorFlow backend.


K.image_dim_ordering: tf
Loaded vgg16 model


In [2]:
# Turn off training on base model layers
for layer in base_model.layers:
    layer.trainable = False
    

# Add on new fully connected layers for the output classes.
# x = Dense(1024, activation='relu')(base_model.get_layer('fc2').output)
# x = Dropout(0.5)(x)
# predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)
base_model_last = base_model.get_layer('flatten').output
x = Dense(4096, activation='relu', name='fc1')(base_model_last)
x = Dense(4096, activation='relu', name='fc2')(x)
predictions = Dense(N_CLASSES, activation='softmax', name='predictions')(x)




model = Model(input=base_model.input, output=predictions)

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [3]:
# Show some debug output
print (base_model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [3]:
# Show some debug output
print (model.summary())

print ('Trainable weights')
#model.save_weights('catdog_pretrain.h5')
#print (model.trainable_weights)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [ ]:
model.load_weights('catdog_pretrain_nf.h5') 
train_data
model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))

In [19]:
model.load_weights('catdog_pretrain_nf.h5') 
# Data generators for feeding training/testing images to the model.
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,  # this is the target directory
        target_size=IMSIZE,  # all images will be resized to 299x299 Inception V3 input
        batch_size=32,
        class_mode='categorical')

model.fit_generator(
        train_generator,
        samples_per_epoch=256,
        nb_epoch=10,
        validation_data=test_generator,
        verbose=2,
        nb_val_samples=256)
model.save_weights('catdog_pretrain_nf.h5')  # always save your weights after training or during training




#img_path = '../data/sport3/validation/hockey/img_2997.jpg'
img_path = '../data/catdog/test/2.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)
#classes= model.predict_classes(x)
#print('Classes:', classes)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/10
9s - loss: 0.2718 - acc: 0.8672 - val_loss: 0.3112 - val_acc: 0.8555
Epoch 2/10
7s - loss: 0.1079 - acc: 0.9648 - val_loss: 0.1906 - val_acc: 0.9102
Epoch 3/10
7s - loss: 0.1572 - acc: 0.9297 - val_loss: 0.1238 - val_acc: 0.9375
Epoch 4/10
7s - loss: 0.1018 - acc: 0.9688 - val_loss: 0.2090 - val_acc: 0.9375
Epoch 5/10
7s - loss: 0.0959 - acc: 0.9570 - val_loss: 0.2894 - val_acc: 0.8945
Epoch 6/10
7s - loss: 0.1786 - acc: 0.9297 - val_loss: 0.1287 - val_acc: 0.9453
Epoch 7/10
7s - loss: 0.0495 - acc: 0.9961 - val_loss: 0.1561 - val_acc: 0.9297
Epoch 8/10
7s - loss: 0.1726 - acc: 0.9258 - val_loss: 0.1697 - val_acc: 0.9258
Epoch 9/10
7s - loss: 0.1359 - acc: 0.9609 - val_loss: 0.1806 - val_acc: 0.9375
Epoch 10/10
7s - loss: 0.1128 - acc: 0.9531 - val_loss: 0.2856 - val_acc: 0.8906
Predicted: [[  1.09740309e-12   1.00000000e+00]]


In [18]:
#model.load_weights('catdog_pretrain.h5') 
#img_path = '../data/sport3/validation/hockey/img_2997.jpg'
img_path = '../data/cat2.jpg'
img_path = '../data/catdog/test/58.jpg'
img = image.load_img(img_path, target_size=IMSIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = inception.preprocess_input(x)

preds = model.predict(x)
print('Predicted:', preds)
#classes= model.predict_classes(x)
#print('Classes:', classes)

Predicted: [[  1.00000000e+00   2.42028775e-11]]
